In [1]:
from pandas import read_pickle, DataFrame
from sklearn.model_selection import train_test_split

In [2]:
data = read_pickle("../data/data_all_features.pkl")

X_train = data.get("X_train")
y_train = data.get("y_train")
X_test = data.get("X_test")
feature_names = data.get("feature_names")
categorical = data.get("categorical")
numerical = data.get("numerical")
id_numeration_train = data.get("id_numeration_train")
id_numeration_test = data.get("id_numeration_test")

In [3]:
def get_visitor_dataframe(X_train, y_train, feature_names):
    visitor_dataframe = DataFrame(columns=feature_names, data=X_train)
    visitor_dataframe["totals_transaction_revenue"] = y_train
    return visitor_dataframe

def add_positive_revenue_column(visitor_dataframe):
    visitor_dataframe["positive_revenue"] = visitor_dataframe["totals_transaction_revenue"] > 0
    visitor_dataframe["positive_revenue"] = visitor_dataframe["positive_revenue"].astype("float")
    return visitor_dataframe

In [4]:
visitor_dataframe = get_visitor_dataframe(X_train, y_train, feature_names)
visitor_dataframe = add_positive_revenue_column(visitor_dataframe)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    visitor_dataframe.drop(columns=["positive_revenue", "totals_transaction_revenue"]), 
    visitor_dataframe["positive_revenue"], 
    test_size = 0.3,
    random_state=1
)

In [6]:
from sklearn.ensemble import RandomForestClassifier
 
forest = RandomForestClassifier(n_estimators=100, n_jobs=5, random_state=1, verbose=True)
#
# Train the mode
#
forest.fit(X_train, y_train.values.ravel())

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    9.9s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:   24.5s finished


RandomForestClassifier(n_jobs=5, random_state=1, verbose=True)

In [7]:
import numpy as np
 
importances = forest.feature_importances_
#
# Sort the feature importance in descending order
#
sorted_indices = np.argsort(importances)[::-1]
 
feat_labels = visitor_dataframe.drop(columns=["positive_revenue", "totals_transaction_revenue"]).columns
 
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,
                            feat_labels[sorted_indices[f]],
                            importances[sorted_indices[f]]))

 1) totals_pageviews               0.265479
 2) totals_hits                    0.254000
 3) weekday                        0.097245
 4) visitNumber                    0.064545
 5) geoNetwork_networkDomain       0.057359
 6) geoNetwork_city                0.017221
 7) device_operatingSystem         0.011426
 8) geoNetwork_metro               0.010475
 9) trafficSource_isTrueDirect     0.010018
10) geoNetwork_region              0.009680
11) geoNetwork_networkDomain_comcast.net 0.008500
12) trafficSource_source_mall.googleplex.com 0.008196
13) totals_newVisits               0.008058
14) device_operatingSystem_Macintosh 0.007464
15) geoNetwork_country             0.006430
16) trafficSource_source           0.006348
17) totals_bounces                 0.005768
18) geoNetwork_country_United States 0.005432
19) geoNetwork_networkDomain_verizon.net 0.005044
20) device_operatingSystem_Windows 0.005018
21) geoNetwork_subContinent_Northern America 0.004955
22) geoNetwork_networkDomain_rr.com 0.00

In [8]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

prediction = forest.predict(X_test)
confusion_matrix(y_test, prediction)
# Display accuracy score
accuracy = accuracy_score(y_test, prediction)
# Display F1 score
f1_score = f1_score(y_test, prediction)
print(f"accuracy: {accuracy}") 
print(f"f1_score: {f1_score}") 

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.2s


accuracy: 0.9868587696204588
f1_score: 0.296223057148507


[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.6s finished


## Demographics

In [ ]:
X_train[0]

In [ ]:
sub_set_feature_names = [
    "geoNetwork_subContinent_@",
    "geoNetwork_subContinent_Caribbean",
    "geoNetwork_subContinent_Central America",
    "geoNetwork_subContinent_Eastern Asia",
    "geoNetwork_subContinent_Northern America",
    "geoNetwork_subContinent_Northern Europe",
    "geoNetwork_subContinent_South America",
    "geoNetwork_subContinent_Western Asia",
    "positive_revenue"
]

visitor_dataframe = get_visitor_dataframe(X_train, y_train, feature_names)
visitor_dataframe = add_positive_revenue_column(visitor_dataframe)

paying_demographic_dataframe = visitor_dataframe[sub_set_feature_names]
paying_demographic_dataframe

In [ ]:
paying_demographic_dataframe[paying_demographic_dataframe["positive_revenue"] == 1]